In [11]:
import sys
import bookutils

In [12]:
from Tracer import Tracer

In [13]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [14]:
class Tracer(object):
    def __init__(self, file=sys.stdout):
        """Trace a block of code, sending logs to file (default: stdout)"""
        self.original_trace_function = None
        self.file = file
        self.log_dict ={}
        self.step=0

        pass

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        # print(*objects, sep=sep, end=end, file=self.file, flush=True)
        # self.logger = self.logger + "\n" + " ".join(objects)
        current_step=""
        for o in objects:
            current_step += repr(o)
        self.log_dict[self.step] = current_step
        self.step += 1

    def traceit(self, frame, event, arg):
        """Tracing function. To be overridden in subclasses."""
        self.log(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)

    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own __exit__() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at begin of `with` block. Turn tracing off."""
        sys.settrace(self.original_trace_function)
        self.get_log()

    def get_log(self):
        print(self.log_dict)

In [15]:
with Tracer():
    remove_html_markup("<b>bold</b>")




{0: "'call'1'remove_html_markup'{'s': '<b>bold</b>'}", 1: "'line'2'remove_html_markup'{'s': '<b>bold</b>'}", 2: "'line'3'remove_html_markup'{'s': '<b>bold</b>', 'tag': False}", 3: "'line'4'remove_html_markup'{'s': '<b>bold</b>', 'tag': False, 'quote': False}", 4: "'line'6'remove_html_markup'{'s': '<b>bold</b>', 'tag': False, 'quote': False, 'out': ''}", 5: "'line'7'remove_html_markup'{'s': '<b>bold</b>', 'tag': False, 'quote': False, 'out': '', 'c': '<'}", 6: "'line'8'remove_html_markup'{'s': '<b>bold</b>', 'tag': False, 'quote': False, 'out': '', 'c': '<'}", 7: "'line'6'remove_html_markup'{'s': '<b>bold</b>', 'tag': True, 'quote': False, 'out': '', 'c': '<'}", 8: "'line'7'remove_html_markup'{'s': '<b>bold</b>', 'tag': True, 'quote': False, 'out': '', 'c': 'b'}", 9: "'line'9'remove_html_markup'{'s': '<b>bold</b>', 'tag': True, 'quote': False, 'out': '', 'c': 'b'}", 10: "'line'11'remove_html_markup'{'s': '<b>bold</b>', 'tag': True, 'quote': False, 'out': '', 'c': 'b'}", 11: "'line'13're